In [7]:
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander

## constants

In [1]:
group_number = 4
uri = f'radio://0/{group_number}/2M'
camera = 2
log_period_ms = 50
side_step = 0.05
forward_step = 0.1

## testing camera

In [21]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Compute
    cv2.imshow('frame', frame)    

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()
del cap

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to tar

## helper functions

In [10]:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)
        
def ascend_and_hover(cf):
    print('ascending')
            
    # Ascend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)

    # Hover at 1 meter:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    
def hover_and_descend(cf):
    print('descending')
    
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
        time.sleep(0.1)
    
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
        
def position_callback(timestamp, data, log_conf):
    global x_cur, y_cur, z_cur
    x_cur = data['kalman.stateX']
    y_cur = data['kalman.stateY']
    z_cur = data['kalman.stateZ']
    
# Sort through contours in the image
def findGreatestContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index

# loop

In [40]:
x_cur = 0.0
y_cur = 0.0
z_cur = 0.0

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('FAILED! FAILED! FAILED!')
else:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        cf = scf.cf
        
        # set up logging
        log_conf = LogConfig(name='Position', period_in_ms=log_period_ms)
        log_conf.add_variable('kalman.stateX', 'float')
        log_conf.add_variable('kalman.stateY', 'float')
        log_conf.add_variable('kalman.stateZ', 'float')

        scf.cf.log.add_config(log_conf)
        log_conf.data_received_cb.add_callback(position_callback)
        log_conf.start()
        
        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        # capture the video
        cap = cv2.VideoCapture(camera)
        
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False
        
        # start to hover
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        while cap.isOpened() and not exit_loop:
            ok, frame = cap.read()
            
            if not ok:
                print('no frame!')
                break
            
            lb1 = (145, 35, 75)
            ub1 = (180, 255, 255)
            lb2 = (0, 75, 75)
            ub2 = (30, 255, 255)

            # Perform contour detection on the input frame.
            hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # Compute mask of red obstacles in either color range.
            mask1 = cv2.inRange(hsv1, lb1, ub1)
            mask2 = cv2.inRange(hsv2, lb2, ub2)
            # Combine the masks.
            mask = cv2.bitwise_or(mask1, mask2)
            
            # Compute
            cv2.imshow('mask', mask)    

            # Hit q to quit.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
            contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            area, idx = findGreatestContour(contours)
#             print(f"areqqa: {area}")
            
            if area > 2000:
                # adjust position
                largest_contour = contours[idx]
#                 print("largest contour", largest_contour)
#                 left_edge = largest_contour[0,1]
#                 right_edge = largest_contour[2,1]
                left_edge = np.amin(largest_contour[:,:,0])
                right_edge = np.amax(largest_contour[:,:,0])
                
                deviation = 0.1
                
                l_bound = mask.shape[1] * (0.5 - deviation)
                c_bound = mask.shape[1] * 0.5
                r_bound = mask.shape[1] * (0.5 + deviation)
                
                print(f'width: {mask.shape[1]}, left edge: {left_edge}, right edge: {right_edge}')
                
                if right_edge < l_bound or left_edge > r_bound:
                    # go forward
                    print(f"contour not in bound; going forward to {x_cur + forward_step}, y_cur = {y_cur}")
                    cf.commander.send_position_setpoint(x_cur + forward_step, y_cur, 1, 0)
                    time.sleep(0.1)
                else:
                    mass_l = np.sum(mask[:, int(l_bound):int(c_bound)])
                    mass_r = np.sum(mask[:, int(c_bound):int(r_bound)])
                    print(f"left mass = {mass_l}, right mass = {mass_r}")
                    
                    if mass_l < mass_r:
                        # go left
                        print(f"going left to {y_cur + side_step}, x_cur = {x_cur}")
                        cf.commander.send_position_setpoint(x_cur, y_cur + side_step, 1, 0)
                        time.sleep(0.1)
                    else:
                        # go right
                        print(f"going right to {y_cur - side_step}, x_cur = {x_cur}")
                        cf.commander.send_position_setpoint(x_cur, y_cur - side_step, 1, 0)
                        time.sleep(0.1)
            else:
                # go forward
                print(f"small area; going forward to {x_cur + forward_step}, y_cur = {y_cur}")
                cf.commander.send_position_setpoint(x_cur + forward_step, y_cur, 1, 0)
                time.sleep(0.1)
            
#             mass_l = np.sum(mask[:, :mask.shape[1]//4])
#             mass_c = np.sum(mask[:, mask.shape[1]//4:mask.shape[1]*3//4])
#             mass_r = np.sum(mask[:, mask.shape[1]*3//4:])
#             print(mass_l, mass_c, mass_r)
            
        cap.release()
        
        # descend
        hover_and_descend(cf)

cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...


Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'

Crazyflies found:
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/21/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/4/2M
radio://0/7/2M
radio://0/21/2M
radio://0/4/2M
radio://0/4/2M
radio://0/7/2M


Error no LogEntry to handle id=1
Error no LogEntry to handle id=1


Initializing PID Controller
Waiting for estimator to find position...
999.9999885708385 999.9999885709431 999.9997719376697
999.9999885708385 999.9999885709431 999.9997719376697
999.9999890500931 999.999989046034 999.999775645847
999.9999890500931 999.999989046034 999.999775645847
999.9999894338134 999.9999894326656 999.999775645847
999.9999894338134 999.9999894326656 999.9997905961791
999.9999894338134 999.9999894326656 999.9997905961791
999.9999894338134 999.9999894326656 999.9997905961791
999.9999894338134 999.9999894326656 999.9997905961791
1.5777850421727635e-06 1.5954392438288778e-06 3.536090662237257e-05
ascending


QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to target thread (0x562079dfc890)

QObject::moveToThread: Current thread (0x562079dfc890) is not the object's thread (0x56207acdd390).
Cannot move to tar

small area; going forward to 2.8043354511260987, y_cur = -5.752331733703613
width: 640, left edge: 455, right edge: 639
contour not in bound; going forward to 2.811246728897095, y_cur = -6.975977897644043
width: 640, left edge: 79, right edge: 300
left mass = 555900, right mass = 83895
going right to -7.043546485900879, x_cur = 2.917437791824341
width: 640, left edge: 392, right edge: 545
contour not in bound; going forward to 3.0209668636322022, y_cur = -7.350038528442383
width: 640, left edge: 0, right edge: 189
contour not in bound; going forward to 2.835970735549927, y_cur = -7.991307258605957
width: 640, left edge: 356, right edge: 428
left mass = 11985, right mass = 647445
going left to -8.21667022705078, x_cur = 2.8085548877716064
small area; going forward to 2.7628665924072267, y_cur = -8.347067832946777
width: 640, left edge: 320, right edge: 639
left mass = 257295, right mass = 1040145
going left to -8.560739707946777, x_cur = 2.749485969543457
width: 640, left edge: 0, right